# An end-to-end workflow to efficiently compress and deploy DNN classifiers on SoC/FPGA.

## DNN training and compression

In [11]:
# Import libraries

import os
import csv
import numpy as np
from numpy import array
import time
import glob

import tensorflow as tf 

from tensorflow.keras.models import *
from tensorflow.keras.layers import *

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import categorical_crossentropy as logloss
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Lambda
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn import decomposition
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE, MDS
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc, roc_auc_score, r2_score, mean_squared_error, mean_absolute_error

import seaborn as sn
import pandas as pd

import keras_tuner as kt
from qkeras import *

from qkeras import QActivation
from qkeras import QDense, QConv2DBatchnorm


#pip install scikit-image
import skimage.data
import skimage.transform
from skimage import io

import shutil, sys

from itertools import cycle

from keras.applications.vgg16 import preprocess_input

# Custom functions
from src.distillationClassKeras import *

# Plot confusion matrix
from src.confMatrix import *

from src.studentCompression import *
from src.studentOptimization import *
from src.studentOptimization_1D import *
from src.teacherOptimization1D import *
from src.teacherOptimization2D import *
from src.teacherOptimization2D_SOTA import *
from src.studentOptimization2D_SOTA import *

from src.teacherTraining import *
from src.loadDataset import *

from src.config import *


In [12]:
# GPU
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

import tensorflow as tf
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)


GPUs:  1


In [13]:
# Definitions

# Dataset path
ROOT_PATH_2D = "dataset/pestControl"
ROOT_PATH_1D = r'dataset/cluster'


# Class labels 
classLabels_2D = ['0', '1', '2']
classLabels_1D = ['0', '1', '2', '3']

nLabels = len(classLabels_1D)

# Input shape for 2D dataset
ROWS, COLS =  80, 80 

# Input samples for 1D dataset
SAMPLES = 30

# Teacher model
# 0: train teacher from scratch, 1: pre-trained model
TEACHER_OP = 0

# Number of iterations for BO
N_ITERATIONS_TEACHER = 1
N_ITERATIONS_STUDENT = 1

# Type of input -->  1: 1D signal, 2: 2D signal, 3: state-of-the art dataset
D_SIGNAL = 3

### Load dataset 

In [4]:
# Pixel normalization
def normalizationPix(train, test):
    # convert from integers to floats
    train_ = train.astype('float32')
    test_ = test.astype('float32')
    # normalize to range 0-1
    train_ = train_ / 255.0
    test_ = test_ / 255.0
    # return normalized images
    
    return train_, test_

In [5]:
if D_SIGNAL == 1:
    # Load 1D signal dataset
    xTrain, xTest, xTest_df_Final, yTrain, yTest, yTest_Final = loadDataset_1D(ROOT_PATH_1D, nLabels, SAMPLES)

elif D_SIGNAL == 2:
    # Load 2D signal dataset
    images_train, images_validation, images_test, y_train, y_test = loadDataset_2D(ROOT_PATH_2D, classLabels_2D, ROWS, COLS)
else: 
    # CIFAR-10 dataset
    from keras.datasets import cifar10
    (images_train, y_train), (images_test, y_test) = keras.datasets.cifar10.load_data()
    images_train, images_test = normalizationPix(images_train, images_test)
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

### Teacher model optimization

In [6]:
# Grab the best hyperparameters

def bestHPBO_computation(bestHP_BO, CONV_VAR, FC_VAR, UPPER_CONV, UPPER_FC):
    
    bestHP = []
    # Grab hyper-params
    for i in range (1,UPPER_CONV+1):
        bestHP.append(bestHP_BO.get(CONV_VAR + str(i)))
    for j in range (1, UPPER_FC+1):
        bestHP.append(bestHP_BO.get(FC_VAR + str(j)))
   
    print("Best hyper-parameter configuration: ", bestHP)
    return bestHP

In [7]:
# Decide if optimize a teacher architecture or load a pre-trained network as teacher

if TEACHER_OP == 0:
    # optimize teacher architecture
    print("Teacher optimization")
    
    if D_SIGNAL == 1:
        print("1D signal")
        bestHP_BO_teacher = teacherBO_1D(xTrain, xTest, yTrain, yTest)
    elif D_SIGNAL == 2:
        print("2D signal")
        bestHP_BO_teacher = teacherBO(images_train, y_train, images_test, y_test)
        # Grab the best hyperparameters
    else: 
        bestHP_BO_teacher = teacherBO_SOTA(images_train, y_train, images_test, y_test, N_ITERATIONS_TEACHER)
else: 

    # Load pre-trained model
    teacherModel = load_model('models/CNN/teacher_NEW_v2_ok.h5')    
    
    teacherModel.summary()

Teacher optimization
INFO:tensorflow:Reloading Oracle from existing project tuner_teacher/untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from tuner_teacher/untitled_project/tuner0.json
INFO:tensorflow:Oracle triggered exit


2023-07-11 16:52:37.410588: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-11 16:52:37.432046: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2304000000 Hz
2023-07-11 16:52:37.433455: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x146f150 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-07-11 16:52:37.433478: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-07-11 16:52:37.485815: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retu

### Teacher training

In [8]:
# Grab the best hyperparameters for teacher training
if TEACHER_OP == 0:
    if D_SIGNAL == 1:
        lr = bestHP_BO_teacher.get('learning_rate')
        CONV_VAR = 'conv_'
        FC_VAR = 'fc'
        UPPER_CONV = 0
        UPPER_FC = 5

        # Grab best hyperparams
        bestHP_BO_teacher = bestHPBO_computation(bestHP_BO_teacher, CONV_VAR, FC_VAR, UPPER_CONV, UPPER_FC)
        print(bestHP_BO_teacher)
        
        # Train 1D teacher model
        teacherModel = teacherTrainingAfterBPO(bestHP_BO_teacher, xTrain, xTest, yTrain, yTest, lr)
        teacherModel.summary()

        # Save model 1D teacher model
        teacherModel.save("models/teacherFP_1D.h5")

    elif D_SIGNAL == 2:

        lr = bestHP_BO_teacher.get('learning_rate')
        CONV_VAR = 'conv_'
        FC_VAR = 'fc'
        UPPER_CONV = 4
        UPPER_FC = 3
        
        # Grab best hyperparams    
        bestHP_BO_teacher = bestHPBO_computation(bestHP_BO_teacher, CONV_VAR, FC_VAR, UPPER_CONV, UPPER_FC)

        # Train 2D teacher model
        teacherModel = teacherTrainingAfterBPO(bestHP_BO_teacher, images_train, y_train, teacherModel, lr)

        teacherModel.summary()

        # Save model 2D teacher model
        teacherModel.save("models/teacherFP_2D.h5")
    
    elif D_SIGNAL == 3:
        from src.teacherTraining2D_SOTA import teacherTrainingAfterBPO_SOTA

        lr = bestHP_BO_teacher.get('learning_rate')
        CONV_VAR = 'conv_'
        FC_VAR = 'fc'
        UPPER_CONV = 8
        UPPER_FC = 3    

        # Grab best hyperparams    
        bestHP_BO_teacher = bestHPBO_computation(bestHP_BO_teacher, CONV_VAR, FC_VAR, UPPER_CONV, UPPER_FC)
       
        # Train 2D teacher model - SOTA dataset
        teacherModel = teacherTrainingAfterBPO_SOTA(bestHP_BO_teacher, images_train, images_test, y_train, y_test, lr)

        teacherModel.summary()

        # Save model 2D teacher model
        teacherModel.save("models/teacherFP_2D_SOTA.h5")

Best hyper-parameter configuration:  [64, 64, 64, 64, 64, 96, 32, 64, 20, 50, 20]
Tacher SOTA
Model: "teacherCNN_SOTA"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_5 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
relu1 (Activation)           (None, 32, 32, 64)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
relu2 (Activation)           (None, 32, 32, 64)        0         
_______________________

2023-07-11 16:52:38.152038: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 614400000 exceeds 10% of free system memory.
2023-07-11 16:52:38.353653: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/32


2023-07-11 16:52:38.882457: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-07-11 16:52:39.009205: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


  3/391 [..............................] - ETA: 13s - loss: 2.3718 - accuracy: 0.1124  

2023-07-11 16:52:40.006012: W tensorflow/core/common_runtime/bfc_allocator.cc:314] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.
2023-07-11 16:52:40.020733: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 1023.69M (1073414144 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-07-11 16:52:40.020944: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 921.32M (966072832 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory


391/391 [==============================] - 17s 38ms/step - loss: 2.0916 - accuracy: 0.2622 - val_loss: 2.1947 - val_accuracy: 0.1996
Epoch 2/32
391/391 [==============================] - 14s 36ms/step - loss: 1.4105 - accuracy: 0.5230 - val_loss: 1.2318 - val_accuracy: 0.5736
Epoch 3/32
391/391 [==============================] - 14s 36ms/step - loss: 1.1507 - accuracy: 0.6127 - val_loss: 1.1266 - val_accuracy: 0.6163
Epoch 4/32
391/391 [==============================] - 14s 36ms/step - loss: 1.0228 - accuracy: 0.6588 - val_loss: 1.0374 - val_accuracy: 0.6520
Epoch 5/32
391/391 [==============================] - 14s 36ms/step - loss: 0.9291 - accuracy: 0.6917 - val_loss: 0.9473 - val_accuracy: 0.6794
Epoch 6/32
391/391 [==============================] - 14s 36ms/step - loss: 0.8671 - accuracy: 0.7117 - val_loss: 0.9689 - val_accuracy: 0.6726
Epoch 7/32
391/391 [==============================] - 14s 36ms/step - loss: 0.7957 - accuracy: 0.7406 - val_loss: 0.9189 - val_accuracy: 0.6980
Epo

### . Student model optimization

In [9]:
# Bayesian optimization for student architecture
if D_SIGNAL == 1:
    bestHP_BO = studentBO_1D(xTrain, xTest, yTrain, yTest, teacherModel, N_ITERATIONS_STUDENT)
elif D_SIGNAL == 2:
    bestHP_BO = studentBO_2D(images_train, y_train, images_test, y_test, teacherModel, N_ITERATIONS_STUDENT)
elif D_SIGNAL == 3:
    bestHP_BO = studentBO_2D_SOTA(images_train, y_train, images_test, y_test, teacherModel, N_ITERATIONS_STUDENT)

INFO:tensorflow:Reloading Oracle from existing project tuner/untitled_project/oracle.json


/tools/anaconda3/envs/neuralEnv/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Reloading Tuner from tuner/untitled_project/tuner0.json
INFO:tensorflow:Oracle triggered exit


### . Student training

In [14]:
if D_SIGNAL == 1:
    lr = bestHP_BO.get('learning_rate')
    CONV_VAR = 'conv_'
    FC_VAR = 'fc'
    UPPER_CONV = 0
    UPPER_FC = 2

    # Grab best hyperparams
    bestHP_BO_student = bestHPBO_computation(bestHP_BO, CONV_VAR, FC_VAR, UPPER_CONV, UPPER_FC)
    print(bestHP_BO_student)
        
    # Train 1D teacher model
    studentModel = studentCompression_1D(bestHP_BO_student, xTrain, xTest, yTrain, yTest, teacherModel, lr)
    studentModel.summary()

    # Save model 1D teacher model
    #studentModel.save("models/teacherFP_1D.h5")

elif D_SIGNAL == 2:

    CONV_VAR = 'conv_'
    FC_VAR = 'fc'
    UPPER_CONV = 9
    UPPER_FC = 4
    lr = bestHP_BO.get('learning_rate')

    # Grab best hyperparams    
    bestHP_BO_student = bestHPBO_computation(bestHP_BO, CONV_VAR, FC_VAR, UPPER_CONV, UPPER_FC)

    # Training to obtain compressed model
    studentModel = studentCompression(bestHP_BO_student, images_train, y_train, teacherModel, lr)

    # Model summary
    studentModel.summary()

    # Save model 2D teacher model
    #studentModel.save("models/studentModel_2D.h5")
elif D_SIGNAL == 3:
    CONV_VAR = 'conv_'
    FC_VAR = 'fc'
    UPPER_CONV = 8
    UPPER_FC = 3
    lr = bestHP_BO.get('learning_rate')

    # Grab best hyperparams    
    bestHP_BO_student = bestHPBO_computation(bestHP_BO, CONV_VAR, FC_VAR, UPPER_CONV, UPPER_FC)

    # Training to obtain compressed model
    studentModel = studentCompression_2D_SOTA(bestHP_BO_student, images_train, images_test, y_train, y_test, teacherModel, lr)

    # Model summary
    studentModel.summary()

    # Save model 2D teacher model
    #studentModel.save("models/studentModel_2D_SOTA.h5")


Best hyper-parameter configuration:  [2, 5, 3, 10, 7, 9, 2, 10, 5, 5, 5]
Model: "qkeras"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv1 (QConv2DBatchnorm)     (None, 32, 32, 2)         65        
_________________________________________________________________
relu1 (QActivation)          (None, 32, 32, 2)         0         
_________________________________________________________________
conv2 (QConv2DBatchnorm)     (None, 32, 32, 5)         116       
_________________________________________________________________
relu2 (QActivation)          (None, 32, 32, 5)         0         
_________________________________________________________________
pool_0 (MaxPooling2D)        (None, 16, 16, 5)         0         
_____________________________________________________

/tools/anaconda3/envs/neuralEnv/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
2023-07-11 17:03:16.644649: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 614400000 exceeds 10% of free system memory.


Epoch 1/32
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/tools/anaconda3/envs/neuralEnv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_46637/2202798754.py", line 49, in <cell line: 1>
    studentModel = studentCompression_2D_SOTA(bestHP_BO_student, images_train, images_test, y_train, y_test, teacherModel, lr)
  File "/home/ro/kaleido/repo/workflowCompressionML_repo/compressionAndTraining/src/studentCompression.py", line 186, in studentCompression_2D_SOTA
  File "/tools/anaconda3/envs/neuralEnv/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py", line 1100, in fit
    tmp_logs = self.train_function(iterator)
  File "/tools/anaconda3/envs/neuralEnv/lib/python3.9/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/tools/anaconda3/envs/neuralEnv/lib/python3.9/site-packages/tensorflow/python/eager

In [ ]:
# Plot confusion matrix for accuracy evaluation
if D_SIGNAL == 1:
    # 1D signal
    confusionMatrixPlot(studentModel, xTest_df_Final, yTest_Final)

else:
    # 2D signal
    confusionMatrixPlot(studentModel, images_train, y_train)

